In [38]:
import re
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

site = 'https://www.amazon.co.za' #amazon.co.za
headers = ['Name', 'Price', 'Rating', 'Rating Count', 'URL Link']
df = pd.DataFrame(columns = headers)

In [39]:
url = site + '/s?k=samsung&ref=nb_sb_noss' #Site Extension
response = requests.get(url)
soup2 = BeautifulSoup(response.content, 'html.parser')
soup = BeautifulSoup(soup2.prettify(), 'html.parser')
dontstop = True
counter = 0

while dontstop:
    x = soup.find_all('div', class_ = "s-asin")
    count = len(x)
    for i in range(count):
        data = []
    #DATA
        x = soup.find_all('div', class_ = "s-asin")
        items = x[i]
    #NAME
        name = items.find('h2')#.text.strip()
        if name is None:
            name = 'N/A'
        else:
            name = name.text.strip()
        name = str(name)
        if ',' in name:
            name = name.replace(',', ';')
        #print('Name: ', name)
        data.append(name)
    #PRICE
        a = items.find('span', class_ = "a-price")
        '''if a is None:
            a = '0,000.00'
        else:
            a = a.text'''
        if a is None:
            a = items.find('span', class_ = "a-color-base")
            if a is None:
                a = '0,000.00'
            else:
                a = a.text
        else:
            a = a.text
        b = re.findall(r'\d{1,3},?\d{1,3}\.\d{2}', a)
        c = b[0].replace(",", "")
        price = float(c)
        '''if price == 0:
            price = 'N/A'
        else:
            price = "{:.2f}".format(price)'''
        #print('Price: ', price)
        #price = "{:.2f}".format(price)
        data.append(price)
    #RATING
        rating = items.find('span', class_ = "a-icon-alt")#.text.strip()
        if rating is None:
            rating = 'N/A'
        else:
            rating = rating.text.strip()
        rating = str(rating)
        #print('Rating: ', rating)
        for x in rating:
            if x == ',':
                x = ';'
        data.append(rating)
        #RATING_COUNT
        rating_c = items.find('span', class_ = "a-size-base")#.text.strip()
        if rating_c is None:
            rating_c = 'N/A'
        else:
            rating_c = rating_c.text.strip()
            try:
                rating_c = int(rating_c)
            except:
                rating_c = 'N/A'
        rating_c = str(rating_c)
        #print('Rating Count: ', rating_c)
        for x in rating_c:
            if x == ',':
                x = ';'
        data.append(rating_c)
    #URL
        u = items.find('a', href = True)
        if (u == None) or ('#customerReviews' in u) or ('javascript:void' in u):
            item_url = 'N/A'
        else:
            item_url = u['href']
            item_url = site + item_url
        item_url = str(item_url)
        #print('Site: ', item_url)
        for x in item_url:
            if x == ',':
                x = ';'
        data.append(item_url)
        counter = counter + 1
        #print(counter)
        #print('')
    #ABOUT
        '''item_response = requests.get(item_url)
        item_soup2 = BeautifulSoup(item_response.content, 'html.parser')
        item_soup = BeautifulSoup(item_soup2.prettify(), 'html.parser')
        about_item = ""
        item_a = item_soup.find_all('li', class_ = "a-spacing-mini")
        about_count = 0
        if item_a == None:
            about_item = 'N/A'
        else:
            for abouts in item_a:
                about_item = about_item + abouts.text.strip()
                about_count = about_count + 1
                if about_count < len(item_a):
                    about_item = about_item + "; "
        data.append(about_item)''' # TOO LONG TO RUN EACH ITEM
        length = len(df)
        df.loc[length] = data
        
    n = soup.find('a', class_ = "s-pagination-next")
    if n == None:
        dontstop = False
    else:
        nxt = n['href']
        nxt = site + nxt
        url = nxt
        response = requests.get(url)
        soup2 = BeautifulSoup(response.content, 'html.parser')
        soup = BeautifulSoup(soup2.prettify(), 'html.parser')

df

,Name,Price,Rating,Rating Count,URL Link
0,Samsung Galaxy Fit3 Smart Watch; Grey,999.0,4.0 out of 5 stars,15,https://www.amazon.co.za/Samsung-Galaxy-Fit3-S...
1,SAMSUNG A05 64GB DS BLACK LTE,1799.0,4.0 out of 5 stars,5,https://www.amazon.co.za/SAMSUNG-A05-64GB-BLAC...
2,Samsung Galaxy A55 256GB 5G Dual Sim Smartphon...,7424.0,5.0 out of 5 stars,3,https://www.amazon.co.za/Samsung-Galaxy-256GB-...
3,Samsung Galaxy Buds FE Earbuds; Graphite,999.0,4.2 out of 5 stars,9,https://www.amazon.co.za/Samsung-Galaxy-Buds-E...
4,Samsung Galaxy S24 Ultra 12GB RAM 256GB Storag...,22696.0,4.1 out of 5 stars,5,https://www.amazon.co.za/Samsung-Galaxy-Storag...
...,...,...,...,...,...
301,SAMSUNG Watch Ultra Titanium White with Buds3,16056.2,N/A,N/A,https://www.amazon.co.za/SAMSUNG-Watch-Ultra-T...
302,Peachz 4 in 1 Retractable Car Charger All Meta...,599.0,5.0 out of 5 stars,1,https://www.amazon.co.za/Peachz-Retractable-Ch...
303,Ein Korpusbasierter Vergleich der Newsrooms vo...,1866.0,N/A,N/A,https://www.amazon.co.za/Korpusbasierter-Vergl...
304,Nylon Woven Replacement Watch Band for Men and...,269.0,N/A,N/A,https://www.amazon.co.za/Replacement-Samsung-G...


In [40]:
url = site + '/s?k=huawei&ref=nb_sb_noss' #Site Extension
response = requests.get(url)
soup2 = BeautifulSoup(response.content, 'html.parser')
soup = BeautifulSoup(soup2.prettify(), 'html.parser')
dontstop = True
counter = 0

while dontstop:
    x = soup.find_all('div', class_ = "s-asin")
    count = len(x)
    for i in range(count):
        data = []
    #DATA
        x = soup.find_all('div', class_ = "s-asin")
        items = x[i]
    #NAME
        name = items.find('h2')#.text.strip()
        if name is None:
            name = 'N/A'
        else:
            name = name.text.strip()
        name = str(name)
        if ',' in name:
            name = name.replace(',', ';')
        #print('Name: ', name)
        data.append(name)
    #PRICE
        a = items.find('span', class_ = "a-price")
        '''if a is None:
            a = '0,000.00'
        else:
            a = a.text'''
        if a is None:
            a = items.find('span', class_ = "a-color-base")
            if a is None:
                a = '0,000.00'
            else:
                a = a.text
        else:
            a = a.text
        b = re.findall(r'\d{1,3},?\d{1,3}\.\d{2}', a)
        c = b[0].replace(",", "")
        price = float(c)
        '''if price == 0:
            price = 'N/A'
        else:
            price = "{:.2f}".format(price)'''
        #print('Price: ', price)
        #price = "{:.2f}".format(price)
        data.append(price)
    #RATING
        rating = items.find('span', class_ = "a-icon-alt")#.text.strip()
        if rating is None:
            rating = 'N/A'
        else:
            rating = rating.text.strip()
        rating = str(rating)
        #print('Rating: ', rating)
        for x in rating:
            if x == ',':
                x = ';'
        data.append(rating)
    #RATING_COUNT
        rating_c = items.find('span', class_ = "a-size-base")#.text.strip()
        if rating_c is None:
            rating_c = 'N/A'
        else:
            rating_c = rating_c.text.strip()
            try:
                rating_c = int(rating_c)
            except:
                rating_c = 'N/A'
        rating_c = str(rating_c)
        #print('Rating Count: ', rating_c)
        for x in rating_c:
            if x == ',':
                x = ';'
        data.append(rating_c)
    #URL
        u = items.find('a', href = True)
        if (u == None) or ('#customerReviews' in u) or ('javascript:void' in u):
            item_url = 'N/A'
        else:
            item_url = u['href']
            item_url = site + item_url
        item_url = str(item_url)
        #print('Site: ', item_url)
        for x in item_url:
            if x == ',':
                x = ';'
        data.append(item_url)
        counter = counter + 1
        #print(counter)
        #print('')
    #ABOUT
        '''item_response = requests.get(item_url)
        item_soup2 = BeautifulSoup(item_response.content, 'html.parser')
        item_soup = BeautifulSoup(item_soup2.prettify(), 'html.parser')
        about_item = ""
        item_a = item_soup.find_all('li', class_ = "a-spacing-mini")
        about_count = 0
        if item_a == None:
            about_item = 'N/A'
        else:
            for abouts in item_a:
                about_item = about_item + abouts.text.strip()
                about_count = about_count + 1
                if about_count < len(item_a):
                    about_item = about_item + "; "
        data.append(about_item)''' # TOO LONG TO RUN EACH ITEM
        length = len(df)
        df.loc[length] = data
        
    n = soup.find('a', class_ = "s-pagination-next")
    if n == None:
        dontstop = False
    else:
        nxt = n['href']
        nxt = site + nxt
        url = nxt
        response = requests.get(url)
        soup2 = BeautifulSoup(response.content, 'html.parser')
        soup = BeautifulSoup(soup2.prettify(), 'html.parser')

df

,Name,Price,Rating,Rating Count,URL Link
0,Samsung Galaxy Fit3 Smart Watch; Grey,999.00,4.0 out of 5 stars,15,https://www.amazon.co.za/Samsung-Galaxy-Fit3-S...
1,SAMSUNG A05 64GB DS BLACK LTE,1799.00,4.0 out of 5 stars,5,https://www.amazon.co.za/SAMSUNG-A05-64GB-BLAC...
2,Samsung Galaxy A55 256GB 5G Dual Sim Smartphon...,7424.00,5.0 out of 5 stars,3,https://www.amazon.co.za/Samsung-Galaxy-256GB-...
3,Samsung Galaxy Buds FE Earbuds; Graphite,999.00,4.2 out of 5 stars,9,https://www.amazon.co.za/Samsung-Galaxy-Buds-E...
4,Samsung Galaxy S24 Ultra 12GB RAM 256GB Storag...,22696.00,4.1 out of 5 stars,5,https://www.amazon.co.za/Samsung-Galaxy-Storag...
...,...,...,...,...,...
607,Toni Merge Case for Huawei Nova 11 Pro; Smokey...,249.00,N/A,N/A,https://www.amazon.co.za/Toni-Merge-Huawei-Smo...
608,Global Tech Parts Huawei Y7 Prime 2018 (LDN-L0...,369.99,N/A,N/A,https://www.amazon.co.za/Huawei-Y7-Prime-2018-...
609,Military Style Smartphone Case With Camera Cov...,120.00,N/A,N/A,https://www.amazon.co.za/Military-Style-Smartp...
610,Flip Wallet Leather Cover Compatible with Huaw...,179.00,N/A,N/A,https://www.amazon.co.za/Leather-Compatible-Hu...


In [41]:
url = site + '/s?k=sony&ref=nb_sb_noss' #Site Extension
response = requests.get(url)
soup2 = BeautifulSoup(response.content, 'html.parser')
soup = BeautifulSoup(soup2.prettify(), 'html.parser')
dontstop = True
counter = 0

while dontstop:
    x = soup.find_all('div', class_ = "s-asin")
    count = len(x)
    for i in range(count):
        data = []
    #DATA
        x = soup.find_all('div', class_ = "s-asin")
        items = x[i]
    #NAME
        name = items.find('h2')#.text.strip()
        if name is None:
            name = 'N/A'
        else:
            name = name.text.strip()
        name = str(name)
        if ',' in name:
            name = name.replace(',', ';')
        #print('Name: ', name)
        data.append(name)
    #PRICE
        a = items.find('span', class_ = "a-price")
        '''if a is None:
            a = '0,000.00'
        else:
            a = a.text'''
        if a is None:
            a = items.find('span', class_ = "a-color-base")
            if a is None:
                a = '0,000.00'
            else:
                a = a.text
        else:
            a = a.text
        b = re.findall(r'\d{1,3},?\d{1,3}\.\d{2}', a)
        c = b[0].replace(",", "")
        price = float(c)
        '''if price == 0:
            price = 'N/A'
        else:
            price = "{:.2f}".format(price)'''
        #print('Price: ', price)
        #price = "{:.2f}".format(price)
        data.append(price)
    #RATING
        rating = items.find('span', class_ = "a-icon-alt")#.text.strip()
        if rating is None:
            rating = 'N/A'
        else:
            rating = rating.text.strip()
        rating = str(rating)
        #print('Rating: ', rating)
        for x in rating:
            if x == ',':
                x = ';'
        data.append(rating)
    #RATING_COUNT
        rating_c = items.find('span', class_ = "a-size-base")#.text.strip()
        if rating_c is None:
            rating_c = 'N/A'
        else:
            rating_c = rating_c.text.strip()
            try:
                rating_c = int(rating_c)
            except:
                rating_c = 'N/A'
        rating_c = str(rating_c)
        #print('Rating Count: ', rating_c)
        for x in rating_c:
            if x == ',':
                x = ';'
        data.append(rating_c)
    #URL
        u = items.find('a', href = True)
        if (u == None) or ('#customerReviews' in u) or ('javascript:void' in u):
            item_url = 'N/A'
        else:
            item_url = u['href']
            item_url = site + item_url
        item_url = str(item_url)
        #print('Site: ', item_url)
        for x in item_url:
            if x == ',':
                x = ';'
        data.append(item_url)
        counter = counter + 1
        #print(counter)
        #print('')
    #ABOUT
        '''item_response = requests.get(item_url)
        item_soup2 = BeautifulSoup(item_response.content, 'html.parser')
        item_soup = BeautifulSoup(item_soup2.prettify(), 'html.parser')
        about_item = ""
        item_a = item_soup.find_all('li', class_ = "a-spacing-mini")
        about_count = 0
        if item_a == None:
            about_item = 'N/A'
        else:
            for abouts in item_a:
                about_item = about_item + abouts.text.strip()
                about_count = about_count + 1
                if about_count < len(item_a):
                    about_item = about_item + "; "
        data.append(about_item)''' # TOO LONG TO RUN EACH ITEM
        length = len(df)
        df.loc[length] = data
        
    n = soup.find('a', class_ = "s-pagination-next")
    if n == None:
        dontstop = False
    else:
        nxt = n['href']
        nxt = site + nxt
        url = nxt
        response = requests.get(url)
        soup2 = BeautifulSoup(response.content, 'html.parser')
        soup = BeautifulSoup(soup2.prettify(), 'html.parser')

df

,Name,Price,Rating,Rating Count,URL Link
0,Samsung Galaxy Fit3 Smart Watch; Grey,999.00,4.0 out of 5 stars,15,https://www.amazon.co.za/Samsung-Galaxy-Fit3-S...
1,SAMSUNG A05 64GB DS BLACK LTE,1799.00,4.0 out of 5 stars,5,https://www.amazon.co.za/SAMSUNG-A05-64GB-BLAC...
2,Samsung Galaxy A55 256GB 5G Dual Sim Smartphon...,7424.00,5.0 out of 5 stars,3,https://www.amazon.co.za/Samsung-Galaxy-256GB-...
3,Samsung Galaxy Buds FE Earbuds; Graphite,999.00,4.2 out of 5 stars,9,https://www.amazon.co.za/Samsung-Galaxy-Buds-E...
4,Samsung Galaxy S24 Ultra 12GB RAM 256GB Storag...,22696.00,4.1 out of 5 stars,5,https://www.amazon.co.za/Samsung-Galaxy-Storag...
...,...,...,...,...,...
889,Sony a Comedy-Drama in Three Acts,748.00,N/A,N/A,https://www.amazon.co.za/Sony-Comedy-Drama-Thr...
890,La guía Friedman Archives para la Sony A7 III,2385.00,5.0 out of 5 stars,N/A,https://www.amazon.co.za/gu%C3%ADa-Friedman-Ar...
891,Bloodborne Official Artworks,1590.00,4.7 out of 5 stars,N/A,https://www.amazon.co.za/Bloodborne-Official-A...
892,Phanerozoic II: Mesozoic | Cenozoic,737.09,4.8 out of 5 stars,N/A,https://www.amazon.co.za/Phanerozoic-II-Mesozo...


In [42]:
url = site + '/s?k=lg&ref=nb_sb_noss' #Site Extension
response = requests.get(url)
soup2 = BeautifulSoup(response.content, 'html.parser')
soup = BeautifulSoup(soup2.prettify(), 'html.parser')
dontstop = True
counter = 0

while dontstop:
    x = soup.find_all('div', class_ = "s-asin")
    count = len(x)
    for i in range(count):
        data = []
    #DATA
        x = soup.find_all('div', class_ = "s-asin")
        items = x[i]
    #NAME
        name = items.find('h2')#.text.strip()
        if name is None:
            name = 'N/A'
        else:
            name = name.text.strip()
        name = str(name)
        if ',' in name:
            name = name.replace(',', ';')
        #print('Name: ', name)
        data.append(name)
    #PRICE
        a = items.find('span', class_ = "a-price")
        '''if a is None:
            a = '0,000.00'
        else:
            a = a.text'''
        if a is None:
            a = items.find('span', class_ = "a-color-base")
            if a is None:
                a = '0,000.00'
            else:
                a = a.text
        else:
            a = a.text
        b = re.findall(r'\d{1,3},?\d{1,3}\.\d{2}', a)
        c = b[0].replace(",", "")
        price = float(c)
        '''if price == 0:
            price = 'N/A'
        else:
            price = "{:.2f}".format(price)'''
        #print('Price: ', price)
        #price = "{:.2f}".format(price)
        data.append(price)
    #RATING
        rating = items.find('span', class_ = "a-icon-alt")#.text.strip()
        if rating is None:
            rating = 'N/A'
        else:
            rating = rating.text.strip()
        rating = str(rating)
        #print('Rating: ', rating)
        for x in rating:
            if x == ',':
                x = ';'
        data.append(rating)
    #RATING_COUNT
        rating_c = items.find('span', class_ = "a-size-base")#.text.strip()
        if rating_c is None:
            rating_c = 'N/A'
        else:
            rating_c = rating_c.text.strip()
            try:
                rating_c = int(rating_c)
            except:
                rating_c = 'N/A'
        rating_c = str(rating_c)
        #print('Rating Count: ', rating_c)
        for x in rating_c:
            if x == ',':
                x = ';'
        data.append(rating_c)
    #URL
        u = items.find('a', href = True)
        if (u == None) or ('#customerReviews' in u) or ('javascript:void' in u):
            item_url = 'N/A'
        else:
            item_url = u['href']
            item_url = site + item_url
        item_url = str(item_url)
        #print('Site: ', item_url)
        for x in item_url:
            if x == ',':
                x = ';'
        data.append(item_url)
        counter = counter + 1
        #print(counter)
        #print('')
    #ABOUT
        '''item_response = requests.get(item_url)
        item_soup2 = BeautifulSoup(item_response.content, 'html.parser')
        item_soup = BeautifulSoup(item_soup2.prettify(), 'html.parser')
        about_item = ""
        item_a = item_soup.find_all('li', class_ = "a-spacing-mini")
        about_count = 0
        if item_a == None:
            about_item = 'N/A'
        else:
            for abouts in item_a:
                about_item = about_item + abouts.text.strip()
                about_count = about_count + 1
                if about_count < len(item_a):
                    about_item = about_item + "; "
        data.append(about_item)''' # TOO LONG TO RUN EACH ITEM
        length = len(df)
        df.loc[length] = data
        
    n = soup.find('a', class_ = "s-pagination-next")
    if n == None:
        dontstop = False
    else:
        nxt = n['href']
        nxt = site + nxt
        url = nxt
        response = requests.get(url)
        soup2 = BeautifulSoup(response.content, 'html.parser')
        soup = BeautifulSoup(soup2.prettify(), 'html.parser')

df

,Name,Price,Rating,Rating Count,URL Link
0,Samsung Galaxy Fit3 Smart Watch; Grey,999.00,4.0 out of 5 stars,15,https://www.amazon.co.za/Samsung-Galaxy-Fit3-S...
1,SAMSUNG A05 64GB DS BLACK LTE,1799.00,4.0 out of 5 stars,5,https://www.amazon.co.za/SAMSUNG-A05-64GB-BLAC...
2,Samsung Galaxy A55 256GB 5G Dual Sim Smartphon...,7424.00,5.0 out of 5 stars,3,https://www.amazon.co.za/Samsung-Galaxy-256GB-...
3,Samsung Galaxy Buds FE Earbuds; Graphite,999.00,4.2 out of 5 stars,9,https://www.amazon.co.za/Samsung-Galaxy-Buds-E...
4,Samsung Galaxy S24 Ultra 12GB RAM 256GB Storag...,22696.00,4.1 out of 5 stars,5,https://www.amazon.co.za/Samsung-Galaxy-Storag...
...,...,...,...,...,...
973,RIVERWELD ESAB L-TEC PT-31 JG-40 WSD-LG40 PT-3...,567.36,4.3 out of 5 stars,665,https://www.amazon.co.za/RIVERWELD-PT-31-WSD-L...
974,Smithsonian Word Search Natural History: Earth...,481.00,4.7 out of 5 stars,N/A,https://www.amazon.co.za/Smithsonian-Word-Sear...
975,ALANES 30X Zoom Magnifier Clip-On Mobile Phone...,199.00,N/A,N/A,https://www.amazon.co.za/ALANES-Magnifier-Clip...
976,Finish 24s Auto Dishwasher Tablets; All in 1 M...,159.99,N/A,N/A,https://www.amazon.co.za/Finish-Dishwasher-Tab...


In [44]:
df = df.map(lambda x: np.nan if x in [0, np.nan, 'N/A'] else x)

In [45]:
df

,Name,Price,Rating,Rating Count,URL Link
0,Samsung Galaxy Fit3 Smart Watch; Grey,999.00,4.0 out of 5 stars,15,https://www.amazon.co.za/Samsung-Galaxy-Fit3-S...
1,SAMSUNG A05 64GB DS BLACK LTE,1799.00,4.0 out of 5 stars,5,https://www.amazon.co.za/SAMSUNG-A05-64GB-BLAC...
2,Samsung Galaxy A55 256GB 5G Dual Sim Smartphon...,7424.00,5.0 out of 5 stars,3,https://www.amazon.co.za/Samsung-Galaxy-256GB-...
3,Samsung Galaxy Buds FE Earbuds; Graphite,999.00,4.2 out of 5 stars,9,https://www.amazon.co.za/Samsung-Galaxy-Buds-E...
4,Samsung Galaxy S24 Ultra 12GB RAM 256GB Storag...,22696.00,4.1 out of 5 stars,5,https://www.amazon.co.za/Samsung-Galaxy-Storag...
...,...,...,...,...,...
973,RIVERWELD ESAB L-TEC PT-31 JG-40 WSD-LG40 PT-3...,567.36,4.3 out of 5 stars,665,https://www.amazon.co.za/RIVERWELD-PT-31-WSD-L...
974,Smithsonian Word Search Natural History: Earth...,481.00,4.7 out of 5 stars,NaN,https://www.amazon.co.za/Smithsonian-Word-Sear...
975,ALANES 30X Zoom Magnifier Clip-On Mobile Phone...,199.00,NaN,NaN,https://www.amazon.co.za/ALANES-Magnifier-Clip...
976,Finish 24s Auto Dishwasher Tablets; All in 1 M...,159.99,NaN,NaN,https://www.amazon.co.za/Finish-Dishwasher-Tab...


In [101]:
dup = df[df.duplicated(subset = ['URL Link', 'Price', 'Name'])]
dup = dup.sort_values(by = 'URL Link')
dup

,Name,Price,Rating,Rating Count,URL Link
292,Body Glove Samsung Galaxy A05S Astrx Case-Black,239.00,NaN,NaN,https://www.amazon.co.za/Body-Glove-Samsung-Ga...
388,65W 3 Port Super Fast Charger with USB & Type-...,227.00,3.0 out of 5 stars,2,https://www.amazon.co.za/Charger-Adapter-Compa...
240,Sam Galaxy A34 5G EE 128-6-5G-gy Sams Galaxy A...,5499.00,4.6 out of 5 stars,248,https://www.amazon.co.za/Galaxy-128-6-5G-gy-Sa...
451,HUAWEI Band 9; Comfortable All-Day Wearing; Sc...,1999.00,4.4 out of 5 stars,NaN,https://www.amazon.co.za/HUAWEI-Comfortable-Sc...
968,Megadream Android Gamepad Controller; Wireless...,300.00,3.3 out of 5 stars,636,https://www.amazon.co.za/Megadream-Controller-...
288,PanzerGlass™ | Samsung Galaxy A55 | Screen Pro...,199.00,NaN,NaN,https://www.amazon.co.za/PanzerGlassTM-Samsung...
457,Penergy QC 3.0 3A Quick Adaptive Fast Charger ...,114.45,NaN,NaN,https://www.amazon.co.za/Penergy-Adaptive-Char...
753,Penergy QC 3.0 3A Quick Adaptive Fast Charger ...,114.45,NaN,NaN,https://www.amazon.co.za/Penergy-Adaptive-Char...
790,Penergy 2.1A Charger with Type C USB Charge Ca...,117.95,NaN,NaN,https://www.amazon.co.za/Penergy-Charger-Charg...
207,Samsung Silicone Mobile Case for Samsung Galax...,299.00,4.2 out of 5 stars,191,https://www.amazon.co.za/Samsung-Silicone-Mobi...


In [99]:
df[df['URL Link'] == dup.iat[7, 4]]

,Name,Price,Rating,Rating Count,URL Link
169,Penergy QC 3.0 3A Quick Adaptive Fast Charger ...,114.45,NaN,NaN,https://www.amazon.co.za/Penergy-Adaptive-Char...
457,Penergy QC 3.0 3A Quick Adaptive Fast Charger ...,114.45,NaN,NaN,https://www.amazon.co.za/Penergy-Adaptive-Char...
753,Penergy QC 3.0 3A Quick Adaptive Fast Charger ...,114.45,NaN,NaN,https://www.amazon.co.za/Penergy-Adaptive-Char...


In [102]:
df_dup = df

In [103]:
df_dup = df_dup.drop_duplicates(subset = ['URL Link', 'Price', 'Name'])

In [104]:
df_dup

,Name,Price,Rating,Rating Count,URL Link
0,Samsung Galaxy Fit3 Smart Watch; Grey,999.00,4.0 out of 5 stars,15,https://www.amazon.co.za/Samsung-Galaxy-Fit3-S...
1,SAMSUNG A05 64GB DS BLACK LTE,1799.00,4.0 out of 5 stars,5,https://www.amazon.co.za/SAMSUNG-A05-64GB-BLAC...
2,Samsung Galaxy A55 256GB 5G Dual Sim Smartphon...,7424.00,5.0 out of 5 stars,3,https://www.amazon.co.za/Samsung-Galaxy-256GB-...
3,Samsung Galaxy Buds FE Earbuds; Graphite,999.00,4.2 out of 5 stars,9,https://www.amazon.co.za/Samsung-Galaxy-Buds-E...
4,Samsung Galaxy S24 Ultra 12GB RAM 256GB Storag...,22696.00,4.1 out of 5 stars,5,https://www.amazon.co.za/Samsung-Galaxy-Storag...
...,...,...,...,...,...
973,RIVERWELD ESAB L-TEC PT-31 JG-40 WSD-LG40 PT-3...,567.36,4.3 out of 5 stars,665,https://www.amazon.co.za/RIVERWELD-PT-31-WSD-L...
974,Smithsonian Word Search Natural History: Earth...,481.00,4.7 out of 5 stars,NaN,https://www.amazon.co.za/Smithsonian-Word-Sear...
975,ALANES 30X Zoom Magnifier Clip-On Mobile Phone...,199.00,NaN,NaN,https://www.amazon.co.za/ALANES-Magnifier-Clip...
976,Finish 24s Auto Dishwasher Tablets; All in 1 M...,159.99,NaN,NaN,https://www.amazon.co.za/Finish-Dishwasher-Tab...


In [8]:
df.to_csv('C:/UNISA/STA1610/AMAZON_SEARCH_RESULTS.csv', index = False)

In [107]:
#End scrape and clean
#Start read and analyze

In [2]:
import pandas as pd
df2 = pd.read_csv('C:/UNISA/STA1610/AMAZON_SEARCH_RESULTS.csv')
df2
#READ FILE IN

,Name,Price,Rating,Rating Count,URL Link
0,Samsung Galaxy Fit3 Smart Watch; Grey,999.00,4.0 out of 5 stars,15.0,https://www.amazon.co.za/Samsung-Galaxy-Fit3-S...
1,SAMSUNG A05 64GB DS BLACK LTE,1799.00,4.0 out of 5 stars,5.0,https://www.amazon.co.za/SAMSUNG-A05-64GB-BLAC...
2,SAMSUNG A05s 64GB SM-A057F/DS LIGHT VIOLET LTE...,2348.00,3.0 out of 5 stars,2.0,https://www.amazon.co.za/SAMSUNG-A05s-SM-A057F...
3,Samsung Galaxy Buds FE Earbuds; Graphite,999.00,4.2 out of 5 stars,9.0,https://www.amazon.co.za/Samsung-Galaxy-Buds-E...
4,Samsung Galaxy A35 128GB 5G Dual Sim Smartphon...,5277.00,5.0 out of 5 stars,2.0,https://www.amazon.co.za/Samsung-Galaxy-128GB-...
...,...,...,...,...,...
1049,Smithsonian Word Search Natural History: Earth...,481.00,4.7 out of 5 stars,NaN,https://www.amazon.co.za/Smithsonian-Word-Sear...
1050,Finish 3 Pack Auto Dishwasher Cleaning Pods; I...,85.00,4.5 out of 5 stars,300.0,https://www.amazon.co.za/Finish-Dishwasher-Cle...
1051,ALANES 30X Zoom Magnifier Clip-On Mobile Phone...,199.00,NaN,NaN,https://www.amazon.co.za/ALANES-Magnifier-Clip...
1052,Finish 24s Auto Dishwasher Tablets; All in 1 M...,159.99,NaN,NaN,https://www.amazon.co.za/Finish-Dishwasher-Tab...


In [87]:
#Items with ratings but no amount of reviews
df2[(df2['Rating'].isnull() == False) & (df2['Rating Count'].isnull() == True)]

,Name,Price,Rating,Rating Count,URL Link
18,Samsung Galaxy A14 4G 64GB Silver,2899.0,4.4 out of 5 stars,NaN,https://www.amazon.co.za/Samsung-Galaxy-A14-64...
20,Samsung Galaxy A14 4G 64GB black,2929.0,4.4 out of 5 stars,NaN,https://www.amazon.co.za/Samsung-Galaxy-A14-64...
29,SAMSUNG Galaxy Watch 6 40mm Bluetooth Smartwat...,4799.0,4.4 out of 5 stars,NaN,https://www.amazon.co.za/SAMSUNG-Bluetooth-Sma...
42,My Samsung Galaxy S III,350.0,4.3 out of 5 stars,NaN,https://www.amazon.co.za/My-Samsung-Galaxy-S-I...
46,Sam Galaxy A34 5G EU-128-6-5G-gy Sams Galaxy A...,5299.0,4.5 out of 5 stars,NaN,https://www.amazon.co.za/Galaxy-EU-128-6-5G-gy...
...,...,...,...,...,...
1041,LG Premium Sketchbook,221.0,4.8 out of 5 stars,NaN,https://www.amazon.co.za/Premium-Sketchbook-Pe...
1043,Finish 250ml Auto Dishwasher Machine Cleaner; ...,69.0,4.6 out of 5 stars,NaN,https://www.amazon.co.za/Finish-Dishwasher-Dis...
1046,CNP Honey Lip Butter (0.5 fl.oz / 15ml) - Prop...,699.0,4.2 out of 5 stars,NaN,https://www.amazon.co.za/CNP-Honey-Butter-fl-o...
1048,Large Print Word Search Puzzles Teal: More Tha...,478.0,4.7 out of 5 stars,NaN,https://www.amazon.co.za/Large-Print-Word-Sear...


In [143]:
#declaring items with a review count 
dfR = df2[df2['Rating Count'].isnull() == False]
dfR

,Name,Price,Rating,Rating Count,URL Link
0,Samsung Galaxy Fit3 Smart Watch; Grey,999.00,4.0 out of 5 stars,15.0,https://www.amazon.co.za/Samsung-Galaxy-Fit3-S...
1,SAMSUNG A05 64GB DS BLACK LTE,1799.00,4.0 out of 5 stars,5.0,https://www.amazon.co.za/SAMSUNG-A05-64GB-BLAC...
2,SAMSUNG A05s 64GB SM-A057F/DS LIGHT VIOLET LTE...,2348.00,3.0 out of 5 stars,2.0,https://www.amazon.co.za/SAMSUNG-A05s-SM-A057F...
3,Samsung Galaxy Buds FE Earbuds; Graphite,999.00,4.2 out of 5 stars,9.0,https://www.amazon.co.za/Samsung-Galaxy-Buds-E...
4,Samsung Galaxy A35 128GB 5G Dual Sim Smartphon...,5277.00,5.0 out of 5 stars,2.0,https://www.amazon.co.za/Samsung-Galaxy-128GB-...
...,...,...,...,...,...
1028,"Kensington Monitor Screen Privacy Filter 23.6""...",3056.06,1.0 out of 5 stars,1.0,https://www.amazon.co.za/Kensington-Monitor-Sc...
1042,Megadream Android Gamepad Controller; Wireless...,300.00,3.3 out of 5 stars,636.0,https://www.amazon.co.za/Megadream-Controller-...
1045,Finish 80s Auto Dishwasher Tablets; All in 1 M...,279.00,4.4 out of 5 stars,28.0,https://www.amazon.co.za/Finish-Dishwasher-Tab...
1047,RIVERWELD ESAB L-TEC PT-31 JG-40 WSD-LG40 PT-3...,567.36,4.3 out of 5 stars,666.0,https://www.amazon.co.za/RIVERWELD-PT-31-WSD-L...


In [85]:
#items with more than 50 reviews
dfR[dfR['Rating Count'].astype(int) > 50]

,Name,Price,Rating,Rating Count,URL Link
8,Samsung 25W 1 Port GaN Travel Adapter; Black,446.00,4.7 out of 5 stars,322.0,https://www.amazon.co.za/Samsung-Port-Travel-A...
11,Samsung 15W 1 Port Fast Travel Adapter with Ty...,399.00,4.6 out of 5 stars,322.0,https://www.amazon.co.za/Samsung-Travel-Adapte...
14,Samsung EP-DG930IBEGWW USB to Type-C Cable; 1....,199.00,4.3 out of 5 stars,75.0,https://www.amazon.co.za/Samsung-EP-DG930IBEGW...
21,Samsung 25W 1 Port GaN Travel Adapter; White,549.00,4.8 out of 5 stars,373.0,https://www.amazon.co.za/Samsung-Port-Travel-A...
24,Samsung 15W 1 Port Fast Travel Adapter with Ty...,395.00,4.7 out of 5 stars,191.0,https://www.amazon.co.za/Samsung-Travel-Adapte...
...,...,...,...,...,...
1024,Kenu Airframe Ultra with Insta-Grip tech | Uni...,650.82,3.6 out of 5 stars,261.0,https://www.amazon.co.za/Kenu-Airframe-Insta-G...
1027,UPGRADED Lifetime Appliance AGZ72909711 Washpl...,2000.00,4.6 out of 5 stars,709.0,https://www.amazon.co.za/Lifetime-Appliance-AG...
1042,Megadream Android Gamepad Controller; Wireless...,300.00,3.3 out of 5 stars,636.0,https://www.amazon.co.za/Megadream-Controller-...
1047,RIVERWELD ESAB L-TEC PT-31 JG-40 WSD-LG40 PT-3...,567.36,4.3 out of 5 stars,666.0,https://www.amazon.co.za/RIVERWELD-PT-31-WSD-L...


In [81]:
df2[df2['Price'] > 1000]

,Name,Price,Rating,Rating Count,URL Link
1,SAMSUNG A05 64GB DS BLACK LTE,1799.00,4.0 out of 5 stars,5.0,https://www.amazon.co.za/SAMSUNG-A05-64GB-BLAC...
2,SAMSUNG A05s 64GB SM-A057F/DS LIGHT VIOLET LTE...,2348.00,3.0 out of 5 stars,2.0,https://www.amazon.co.za/SAMSUNG-A05s-SM-A057F...
4,Samsung Galaxy A35 128GB 5G Dual Sim Smartphon...,5277.00,5.0 out of 5 stars,2.0,https://www.amazon.co.za/Samsung-Galaxy-128GB-...
5,SAMSUNG A05 64GB LIGHT GREEN LTE AFA,1799.00,3.4 out of 5 stars,3.0,https://www.amazon.co.za/SAMSUNG-A05-64GB-LIGH...
6,Samsung Galaxy A35 128GB 5G Dual Sim Smartphon...,5473.00,5.0 out of 5 stars,5.0,https://www.amazon.co.za/Samsung-Galaxy-128GB-...
...,...,...,...,...,...
1028,"Kensington Monitor Screen Privacy Filter 23.6""...",3056.06,1.0 out of 5 stars,1.0,https://www.amazon.co.za/Kensington-Monitor-Sc...
1036,Impact du marketing électronique sur l'intenti...,1338.00,NaN,NaN,https://www.amazon.co.za/marketing-%C3%A9lectr...
1037,Impacto do E-Marketing na intenção de compra d...,1283.00,NaN,NaN,https://www.amazon.co.za/Impacto-Marketing-int...
1038,Impatto dell'E-Marketing sull'intenzione di ac...,1336.00,NaN,NaN,https://www.amazon.co.za/Impatto-dellE-Marketi...


In [83]:
#Items going for more than 1000 that have more than 50 reviews
dfR[(dfR['Price'] > 1000) & (dfR['Rating Count'].astype(int) > 50)]

,Name,Price,Rating,Rating Count,URL Link
72,"SAMSUNG Galaxy Tab A9 8.7"" SM-X115 64GB 4GB Fa...",3395.00,4.2 out of 5 stars,81.0,https://www.amazon.co.za/SAMSUNG-SM-X115-Facto...
76,Samsung Galaxy A05s (4GB; 128GB Storage) | 50 ...,2949.00,4.4 out of 5 stars,108.0,https://www.amazon.co.za/Samsung-Galaxy-Storag...
83,Samsung Galaxy Z Fold6 Silicone Stand With Str...,1299.00,2.7 out of 5 stars,60.0,https://www.amazon.co.za/Samsung-Galaxy-Fold6-...
84,Samsung Galaxy A05s (4GB; 128GB Storage) | 50 ...,2899.00,4.4 out of 5 stars,263.0,https://www.amazon.co.za/Samsung-Galaxy-Storag...
98,Pelican Shield - Samsung Galaxy S24 Ultra Case...,1263.65,4.4 out of 5 stars,171.0,https://www.amazon.co.za/Pelican-Shield-Wirele...
...,...,...,...,...,...
985,"LG 22MP410-B Computer Monitor 54.5 cm (21.4"") ...",2257.00,4.3 out of 5 stars,67.0,https://www.amazon.co.za/LG-22MP410-B-Computer...
987,LG 68;6cm/27'' (2560x1440) 27QN880-B 16:9 5ms ...,10731.00,4.5 out of 5 stars,81.0,https://www.amazon.co.za/LG-6cm-2560x1440-27QN...
990,LG 20MK400H-B 19.5 LED Full HD Monitor,1769.00,4.1 out of 5 stars,198.0,https://www.amazon.co.za/LG-20MK400H-B-19-5-Fu...
996,LG Electronics Full HD Monitor 24MP400; 24 Inc...,2804.00,4.3 out of 5 stars,51.0,https://www.amazon.co.za/LG-Electronics-Adjust...


In [147]:
#Average Cost per searched brand
brands = ['Samsung', 'Huawei', 'Sony', 'LG']
SUM = [0.0, 0.0, 0.0, 0.0]
COUNT = [0, 0, 0, 0]
itms = len(df2)
for i in range(itms):
    for x in range(4):
        if brands[x] in df2.at[i, 'Name']:
            SUM[x] = SUM[x] + df2.at[i, 'Price']
            COUNT[x] = COUNT[x] + 1

for x in range(4):
    print(brands[x], ": R ", SUM[x]/COUNT[x])

Samsung : R  2415.7500655737704
Huawei : R  618.3719503546101
Sony : R  9692.995934579438
LG : R  3408.3260144927535


In [140]:
#brands[0] in df2.at[0, 'Name']

In [139]:
#df2.at[0, 'Price']